<a href="https://colab.research.google.com/github/aarav2703/Video-Denoising-Optimization/blob/main/temporal_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Cell 1: Mount Google Drive to access files
from google.colab import drive
drive.mount('/content/drive')

# Cell 2: Install necessary libraries
!pip install moviepy

# Cell 3: Import libraries
import numpy as np
import moviepy.editor as mp

# Cell 4: Define function for temporal filtering (e.g., median filter)
def temporal_filter(frames, filter_size=3):
    filtered_frames = []
    for i in range(len(frames)):
        start_idx = max(0, i - filter_size // 2)
        end_idx = min(len(frames), i + filter_size // 2 + 1)
        filtered_frame = np.median(frames[start_idx:end_idx], axis=0)
        filtered_frames.append(filtered_frame)
    return filtered_frames

# Cell 5: Specify directory where the video is located
video_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/artificial noise adder/'

# Cell 6: Specify the name of the input video file
video_filename = 'pool resort.mp4'

# Cell 7: Process and denoise the video in segments with reduced frame size
video_path = video_directory + video_filename

video_clip = mp.VideoFileClip(video_path)
fps = video_clip.fps
duration = video_clip.duration
segment_duration = 4  # Process 10 seconds of video at a time
num_segments = int(np.ceil(duration / segment_duration))

output_frames = []

for i in range(num_segments):
    start_time = i * segment_duration
    end_time = min((i + 1) * segment_duration, duration)
    segment_clip = video_clip.subclip(start_time, end_time)
    segment_clip_resized = segment_clip.resize((1920,1080))  # Resize frame to reduce memory usage
    segment_frames = [frame for frame in segment_clip_resized.iter_frames()]
    denoised_frames = temporal_filter(segment_frames)
    output_frames.extend(denoised_frames)

# Cell 8: Specify directory where the denoised video will be saved
output_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/'

# Cell 9: Create a new video clip with denoised frames
denoised_video_clip = mp.ImageSequenceClip(output_frames, fps=fps)

# Cell 10: Define output video file path
output_video_path = output_directory + 'pool resort temporal.mp4'

# Cell 11: Write denoised video clip to the output file
denoised_video_clip.write_videofile(output_video_path)

# Cell 12: Provide download link for the denoised video
print("Denoising completed. Download the denoised video:")
print(output_video_path)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


  warnings.warn("Warning: in file %s, "%(self.filename)+



Moviepy - Building video /content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/pool resort temporal.mp4.
Moviepy - Writing video /content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/pool resort temporal.mp4



Moviepy - Done !
Moviepy - video ready /content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/pool resort temporal.mp4
Denoising completed. Download the denoised video:
/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/pool resort temporal.mp4


In [ ]:
import numpy as np
import moviepy.editor as mp

# Cell 4: Define function for motion-compensated temporal filtering
def temporal_filter(frames, motion_frames, filter_size=3):
    filtered_frames = []
    for i in range(len(frames)):
        start_idx = max(0, i - filter_size // 2)
        end_idx = min(len(frames), i + filter_size // 2 + 1)
        filtered_frame = np.median(frames[start_idx:end_idx], axis=0)

        # Apply motion compensation
        if i > 0:
            motion_frame = motion_frames[i - 1]
            filtered_frame += motion_frame

        filtered_frames.append(filtered_frame)
    return filtered_frames

# Cell 5: Specify directory where the video is located
video_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/artificial noise adder/'

# Cell 6: Specify the name of the input video file
video_filename = 'pool resort.mp4'

# Cell 7: Process and denoise the video in segments with reduced frame size
video_path = video_directory + video_filename

video_clip = mp.VideoFileClip(video_path)
fps = video_clip.fps
duration = video_clip.duration
segment_duration = 4  # Process 10 seconds of video at a time
num_segments = int(np.ceil(duration / segment_duration))

output_frames = []
motion_frames = []

for i in range(num_segments):
    start_time = i * segment_duration
    end_time = min((i + 1) * segment_duration, duration)
    segment_clip = video_clip.subclip(start_time, end_time)
    segment_clip_resized = segment_clip.resize((1920,1080))  # Resize frame to reduce memory usage
    segment_frames = [frame for frame in segment_clip_resized.iter_frames()]

    # Motion estimation for the segment
    motion_segment_frames = [np.zeros_like(segment_frames[0])]
    for j in range(1, len(segment_frames)):
        motion_vector = cv2.optflow.calcOpticalFlowSF(
            cv2.cvtColor(segment_frames[j - 1], cv2.COLOR_RGB2GRAY),
            cv2.cvtColor(segment_frames[j], cv2.COLOR_RGB2GRAY),
            layers=3,
            averaging_block_size=2,
            max_flow=4
        )
        motion_frame = np.zeros_like(segment_frames[j])
        motion_frame[:, :, 0] = motion_vector[:, :, 0]
        motion_frame[:, :, 1] = motion_vector[:, :, 1]
        motion_segment_frames.append(motion_frame)
    motion_frames.extend(motion_segment_frames)

    # Apply temporal filtering
    denoised_frames = temporal_filter(segment_frames, motion_segment_frames)
    output_frames.extend(denoised_frames)

# Cell 8: Specify directory where the denoised video will be saved
output_directory = '/content/drive/MyDrive/Colab Notebooks/video denoising media/denoised videos/'

# Cell 9: Create a new video clip with denoised frames
denoised_video_clip = mp.ImageSequenceClip(output_frames, fps=fps)

# Cell 10: Define output video file path
output_video_path = output_directory + 'pool_resort_temporal_motion.mp4'

# Cell 11: Write denoised video clip to the output file
denoised_video_clip.write_videofile(output_video_path)

# Cell 12: Provide download link for the denoised video
print("Denoising completed. Download the denoised video:")
print(output_video_path)
